<a href="https://colab.research.google.com/github/kieranxsomer/HuggingFace_Backup/blob/main/HuggingFace_Backup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hugging Face Backup - Upload your junk from Google Drive
*Quickly upload stuff like merged models to huggingface*

Run cell `1,2,3`

You only need to choose one between `3.1,3.2,and 3.3`

<small>Forked from LYNN - AND NOCRYPT - Literally just yoinked it from this notebook: https://colab.research.google.com/drive/1wEa-tS10h4LlDykd87TF5zzpXIIQoCmq

WE TAKE NO CREDIT ON THIS. FOLLOW NOCRYPTS STUFF HERE: 

[![](https://dcbadge.vercel.app/api/shield/442099748669751297?style=flat)](https://lookup.guru/442099748669751297) [![ko-fi](https://img.shields.io/badge/Ko--fi-F16061?logo=ko-fi&logoColor=white&style=flat)](https://ko-fi.com/nocrypt) [![ko-fi](https://img.shields.io/badge/Patreon-F1465A?logo=patreon&logoColor=white&style=flat)](https://patreon.com/nocrypt) 


<small> You can find updates on NoCrypt's toys: https://discord.gg/touhouai

<small> Earth & Dusk Discord because REASONS: https://discord.gg/5t2kYxt7An


## How to set up the files

What you need to NOW sadly is just replace certain "code" or locations with the locations of where you need to back up to your folder. 

### Double click on the `UPLOAD VIA HUGGINGFACE` section and see the following section:
paths_map 
    
    `Models`  /content/sdw/models/Stable-diffusion
    
    `VAEs`  /content/sdw/models/VAE
    
    `LORAs`  /content/sdw/models/Lora
    
    `Embeddings`  /content/sdw/embeddings
   
    `Hypernetworks`  /content/sdw/models/hypernetworks


You will need to change the `MODELS` section or any of the above sections to whatever folder you're looking to back up.  It's easy as just looking on the side file browser, scrolling through and copying the path of the folder in question. 

Example: `/content/drive/MyDrive/cagliostro-colab-ui` may be your content folder you're looking for, just copy the path and paste it to one of the above sections.


<small> Sadly I don't code. So there's no easier way of doing this LOL. 

### Also Note! 

Make sure you `MOUNT` your google drive, I've patched in `Imjoy-Elfinder` for kicks. - Visual people like this, I know I do.

### MAKE SURE: 

You create a huggingface token, go to [this link](https://huggingface.co/settings/tokens), then `create new token` or copy available token with the `Write` role.

# Google Drive Mount 
<small> If you're NOT using drive, but have a way of uploading your files via command line - use the Imjoy-Elfinder section below. 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Imjoy-Elfinder (Optional)
<small> Useful if you need to check where files are OR you're not using google drive and have uploaded your files already.

In [ ]:
!pip install imjoy-elfinder

In [ ]:
import threading
from google.colab import output
from imjoy_elfinder.app import main

# start imjoy-elfinder server
thread = threading.Thread(target=main, args=[["--root-dir=/content", "--port=8765"]])
thread.start()

# view the 
output.serve_kernel_port_as_iframe(8765, height='800')

In [ ]:
# open imjoy-elfinder in a new tab
output.serve_kernel_port_as_window(8765)

# Upload to Huggingface Hub (FUN WAY)


In [ ]:
#@title 1. Login to Huggingface hub
try:
  hub_ok # My packaged dep didn't contains this (but stil have the folder)
except:
  print("Setting up huggingface_hub...")
  !pip install --force-reinstall -qqqq huggingface_hub
  hub_ok = True
from IPython.display import clear_output
from huggingface_hub import login
clear_output()

#@markdown 1. Of course, you need a Huggingface account first.
#@markdown 2. To create a huggingface token, go to [this link](https://huggingface.co/settings/tokens), then `create new token` or copy available token with the `Write` role.

write_token = "" #@param {type:"string"}
login(write_token, add_to_git_credential=True)

In [ ]:
#@title 2. Setup Repo
from huggingface_hub.utils import validate_repo_id, HfHubHTTPError
from huggingface_hub import HfApi


api = HfApi()
user = api.whoami(write_token)

#@markdown #### If your model repo didn't exist, it will automatically create your repo.
repo_name = "my_merged_models" #@param{type:"string"}
make_this_repo_private_if_not_exist = False #@param{type:"boolean"}
clone_with_git = True #@param{type:"boolean"}

model_repo = user['name']+"/"+repo_name.strip()

validate_repo_id(model_repo)

if repo_name != "":
  try:
      api.create_repo(repo_id=model_repo, 
                      private=make_this_repo_private_if_not_exist)
      print("Model Repo didn't exists, creating repo")
      print("Model Repo: ",model_repo,"created!\n")

  except HfHubHTTPError as e:
      print(f"Model Repo: {model_repo} exists, skipping create repo\n")

if clone_with_git:
  !git lfs install --skip-smudge
  !export GIT_LFS_SKIP_SMUDGE=1
  !git clone https://huggingface.co/{model_repo} /content/{repo_name}

In [ ]:

import os
import sys
import ipywidgets as widgets
from IPython.display import clear_output
from huggingface_hub import HfApi, login
from huggingface_hub.utils import validate_repo_id, HfHubHTTPError
from IPython.utils import capture


paths_map = {
    "Models" : "/content/sdw/models/Stable-diffusion",
    "VAEs" : "/content/sdw/models/VAE",
    "LORAs" : "/content/sdw/models/Lora",
    "Embeddings" : "/content/sdw/embeddings",
    "Hypernetworks" : "/content/sdw/models/hypernetworks",
}

#@title  3.1 Upload via huggingface_hub (Fun way) 
#@markdown ## **How to use this?**<br>
#@markdown 1. Run this cell after you ran the login cell
#@markdown 2. Select model you want to upload (use `ctrl/shift` for multiple selection)
#@markdown 3. Click on upload button
folder = "Models" #@param ["Models", "VAEs", "LORAs", "Embeddings", "Hypernetworks"]
commit_message = "Upload with 🚀🤗 NoCrypt's HUGGING FACE backup - Yoinked by Earth & Dusk"  #@param{type:"string"}

models_path = paths_map[folder]
upload_path = '/content/upload_models'

api = HfApi()
username_repo = user['name']+"/"+repo_name.strip()
validate_repo_id(username_repo)

def get_file_list(path):
  res = []
  for (dir_path, dir_names, file_names) in os.walk(path):
      res.extend(file_names)
  return res
  
selected = widgets.SelectMultiple(
    options=get_file_list(models_path),
    rows=10,
    disabled=False,
)

button = widgets.Button(
    description='Upload',
    disabled=False,
    button_style='success',
    tooltip='Upload to huggingface',
)
dropdown = widgets.Dropdown(
    options=paths_map.keys(),
    value=folder,
    description='Folder',
)

out = widgets.Output()

def on_folder_change(change):
  if change['type'] == 'change' and change['name'] == 'value':
      models_path = paths_map[change["new"]]
      selected.options = get_file_list(models_path)

def upload_it(b):
  with out:
    if selected.value is not None:
      clear_output()
      !mkdir -p {upload_path}
      
      
      #hard link each file
      for selected_model in selected.value:
        if not os.path.exists(os.path.join(upload_path,selected_model)):
          os.link(os.path.join(paths_map[dropdown.value],selected_model),os.path.join(upload_path,selected_model)) #hardlinking to save colab's space
      
      #delete .ipynb_checkpoint
      if os.path.exists(os.path.join(upload_path,".ipynb_checkpoints")):
        !rm {upload_path}/.ipynb_checkpoints
      print("Selected:", ", ".join(selected.value))
      print("Uploading to https://huggingface.co/"+username_repo)
      print("Please wait... Might look stuck, but it's not 👍")

      # Comment this for file based upload
      api.upload_folder(
          folder_path=upload_path,
          repo_id=username_repo,
          commit_message=commit_message
      )

      # Uncomment for file based upload
      # for filename in os.listdir(upload_path):
      #   f = os.path.join(upload_path, filename)
      #   # checking if it is a file
      #   if os.path.isfile(f):
      #     api.upload_file(
      #       path_or_fileobj=f,
      #       path_in_repo=filename,
      #       repo_id=username_repo,
      #       commit_message=commit_message
      #     )
      
      print("Done!")
      #delete hardlink
      !rm -rf {upload_path}/*
    else:
      print("Nothing is selected")
      
dropdown.observe(on_folder_change)
button.on_click(upload_it)
print("Upload target: https://huggingface.co/"+username_repo)
print("👇 Select models you want to upload (use ctrl/shift for multiple selection) ")
display(dropdown,selected,button,out)

# Manual Huggingface Uploads


In [ ]:
#@title 3.2 Upload via huggingface_hub (Manual way)
#@markdown If these are not your folder settings, feel free to just copy the path like you would have for the above "FUN WAY" section.

#@markdown All paths for easier access:<br>
#@markdown /content/sdw/models/Stable-diffusion<br>
#@markdown /content/sdw/models/VAE<br>
#@markdown /content/sdw/models/Lora<br>
#@markdown /content/sdw/embeddings<br>
#@markdown /content/sdw/models/hypernetwork<br>
from huggingface_hub import HfApi
from pathlib import Path

api = HfApi()
file_path = "/content/models/Anything-V3-vae-swapped.safetensors" #@param {type :"string"}
commit_message = "Upload with 🚀🤗 NoCrypt's HUGGING FACE backup - Yoinked by Earth & Dusk" #@param {type :"string"}

if file_path != "":
  path_obj = Path(file_path)
  trained_model = path_obj.parts[-1]

  print(f"Uploading {trained_model} to https://huggingface.co/"+model_repo)
  print(f"Please wait...")

  api.upload_file(
      path_or_fileobj=file_path,
      path_in_repo=trained_model,
      repo_id=model_repo,
      commit_message=commit_message,
  )
  
  print(f"Upload success, located at https://huggingface.co/"+model_repo+"/blob/main/"+trained_model+"\n")

# Upload Via GIT (supposedly more stable)

In [ ]:
#@title 3.3 Upload via GIT (more stable)
%cd /content/
file_path = "/content/models/Anything-V3-vae-swapped.safetensors" #@param {type :"string"}
!ln {file_path} /content/{repo_name}/
#@markdown Set **git commit identity**
email = "your-email" #@param {'type': 'string'}
name = "your-username" #@param {'type': 'string'}
#@markdown Set **commit message**
commit_m = "Upload with 🚀🤗 NoCrypt's HUGGING FACE backup - Yoinked by Earth & Dusk" #@param {'type': 'string'}

%cd /content/{repo_name}

!git lfs install --skip-smudge
!export GIT_LFS_SKIP_SMUDGE=1
!git pull -X theirs

!git lfs install
!huggingface-cli lfs-enable-largefiles .
!git config --global user.email "{email}"
!git config --global user.name "{name}"
!git add .
!git commit -m "{commit_m}"
!git push